# Simple intent recognition and question answering with DeepPavlov

This notebook consists of the [DeepPavlov](https://github.com/deepmipt/DeepPavlov) code snippets. The snippets show how to interact with the text classification models that were specifically developed to be effective when training data is limited. The popular use case scenario for these models is to classify user utterances into one of the FAQ questions and retrieve the corresponding answer (autoFAQ models). As a testbed, we use the students’ FAQ from the [MIPT website](https://mipt.ru/english/). The FAQ contains the most popular first-year students' questions with corresponding answers.
The framework allows you to train models, fine-tune hyperparameters, and test models.

# Requirements

First, install all required packages

In [ ]:
!pip install -q deeppavlov

# Model Description

DeepPavlov contains several text classification models that work well on few training pairs. All the models are based on two major text representations: fastText word embeddings and tf-idf representation. The models described in the separated configuration files under the [config/faq](https://github.com/deepmipt/DeepPavlov/tree/master/deeppavlov/configs/faq) folder. The config file consists of four main sections: **dataset_reader**, **dataset_iterator**, **chainer**, and **train**.

The **dataset_iterator** specifies how to split the data into train, valid, test sets. The **chainer** section of the configuration files contains a pipeline of the required components to interact with the models, i.e. tokenizer, lemmatizer, tf-idf vectorizer, and others. The tokenizer splits a string into tokens, lemmatizer converts all tokens into lemmas. The tf-idf vectorizer transforms the lemmas into tf-idf vectors. The component’s input and output are defined in the **in** and **out** keys correspondingly.

Run the following cell to see a [configuration file](https://github.com/deepmipt/DeepPavlov/blob/master/deeppavlov/configs/faq/tfidf_logreg_en_faq.json) based on logistic regression.

In [ ]:
%load https://raw.githubusercontent.com/deepmipt/DeepPavlov/master/deeppavlov/configs/faq/tfidf_logreg_en_faq.json

# Interacting with the model

The DeepPavlov framework contains several models pre-trained on the aforementioned MIPT FAQ corpus. The files with the pre-trained models defined in the **metadata: download** section of the model's configuration file. You can interact with the model by running it from the command line with **interact** parameter and the name of  the model's configuration file (-d indicates to download all required files). But first, **install** all the model requirements.

In [ ]:
!python -m deeppavlov install tfidf_logreg_en_faq

In [ ]:
!python -m deeppavlov interact tfidf_logreg_en_faq -d

Alternatively, you can **build_model** from the Python code as on the example below. In addition, please make sure that you can navigate the configuration files by using Autocomplete (Tab key) with **configs** module.

In [ ]:
from deeppavlov import configs
from deeppavlov.core.common.file import read_json
from deeppavlov.core.commands.infer import build_model

faq = build_model(configs.faq.tfidf_logreg_en_faq, download = True)
a = faq(["I need help"])
a

# Training the model

You can train a model by running the library with **train** parameter, wherein the model will be trained on the dataset defined in the dataset_reader section of the configuration file. If **metrics** key along with either **validate_best** or **test_best** is defined in the train section, the model will be validated/tested on the corresponding set in the dataset_iterator section.

In [ ]:
!python -m deeppavlov train tfidf_logreg_en_faq

Let's modify the training data and retrain the model.

In [ ]:
%%bash
wget -q http://files.deeppavlov.ai/faq/school/faq_school_en.csv -O faq.csv
echo "What's DeepPavlov?, DeepPavlov is an open-source conversational AI library" >> faq.csv

In [ ]:
from deeppavlov import configs, train_model

model_config = read_json(configs.faq.tfidf_logreg_en_faq)
model_config["dataset_reader"]["data_path"] = "/content/faq.csv"
model_config["dataset_reader"]["data_url"] = None
faq = train_model(model_config)
a = faq(["tell me about DeepPavlov"])
a

# About Us

We are iPavlov, our story started in 2017 when we decided to build a conversational AI framework that on the one hand will contain all required NLP components to build chatbots and on the other hand will be easy to use. Our work resulted in releasing DeepPavlov library. Our lab at MIPT is honored with Facebook AI Academic Partnership and NVIDIA GPU Research Center status. We successfully combine research and extreme coding in our week-long DeepHack.me hackathons — DeepHack.Game, DeepHack.Q&A and DeepHack.RL. We serve a global AI community by organizing NIPS Conversational Challenge to evaluate state-of-the-art techniques in the field of dialog systems and collect open source dialog datasets.

# Useful links

[DeepPavlov repository](https://github.com/deepmipt/DeepPavlov)

[DeepPavlov demo page](https://demo.ipavlov.ai)

[DeepPavlov documentation](https://docs.deeppavlov.ai)